In [ ]:
# importing required modules
from os.path import isfile, isdir
from os import mkdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.signal import savgol_filter
from scipy.stats import pearsonr, spearmanr

In [ ]:
# setting up the correct paths
result_path = "../results/"
data_path = "../data/"

In [ ]:
# setting up folder structure

if isdir(result_path) == False:
    print("[*] Result directory does not exist, building result directory.")
    mkdir(result_path)
    print("[+] DONE")

In [ ]:
# decompress meta_data for plots
xztarfile = data_path+"meta_data.tar.xz"
tarfile = data_path+"meta_data.tar"
if isfile(xztarfile):
    !xz -d -v $xztarfile
if isfile(tarfile):
    !tar -xvf $tarfile -C ../data/

In [ ]:
# decompress growth and fluorescence dataset
xztarfile = data_path+"r2aplus_30degree.tar.xz"
tarfile = data_path+"r2aplus_30degree.tar"
if isfile(xztarfile):
    !xz -d -v $xztarfile
if isfile(tarfile):
    !tar -xvf $tarfile -C ../data/

In [ ]:
# overview
# 470 green
# 570 red
# J23100 == CPL_0017

In [ ]:
'''generate_time_in_minutes_list

    This function takes a dataframe in ../data/ with the measurement timepoints in the first row.
    The measurement time has following format: int h int min. It then calculates the
    minutes of the measurement time as an integer value and appends this value into a
    list.
    
    :param dataframe
        :type pd.DataFrame
    
    :returns minute_time
        :type list[int]

'''
def generate_time_in_minutes_list(dataframe:pd.DataFrame)->list:
    counter = -1
    minute_time = []
    for i in list(dataframe.iloc[0,2:]):
        hours = int(i.split("h")[0])
        minutes = i.split("min")[0].split("h")[-1]
        
        if counter == 0 or counter == -1:
        
            if minutes != ' ':
                minutes = int(minutes)
            else:
                minutes = 0
                    
            time_in_minutes = minutes + (hours * 60)
        
            if int(time_in_minutes) == 0:
                counter += 1
            
            if counter == 0 or counter == -1:
                minute_time.append(time_in_minutes)
            
        else:
            break
            
    return minute_time

In [ ]:
'''extract_content
    
    This function takes a dataframe in ../data/ with a Content column in which
    the 5'UTR identifier reside. In each row there should be values for one 5'UTR identifier
    named in the format: CPL_XXXX where XXXX is a number. The function returns a list of 
    all CPL identifier in the current dataframe.
    
    :param df
        :type pd.DataFrame
        
    :returns content_to_plot
        :type list[str]

'''
def extract_content(df:pd.DataFrame)->list:
    content_to_plot = []
    for cont in df.Content:
        if type(cont) == str:
            if cont.startswith("CPL"):
                
                content_to_plot.append(cont)
    return content_to_plot

In [ ]:
'''extract_datasets
    
    This function takes a dataframe in ../data/ and return the OD, GFP and RFP values
    as pandas dataframes.
    
    :param df
        :type pd.DataFrame
    
    :returns [od_df,green_df,red_df]
        :type list[pd.DataFrame]

'''
def extract_datasets(df:pd.DataFrame)->list:

    od_columns = ["Well","Content"]
    green_columns = ["Well","Content"]
    red_columns = ["Well","Content"]

    for i in df.columns:
        if i.startswith('Blank corrected based on Raw Data (600'):
            od_columns.append(i)
        elif i.startswith(' Blank corrected based on Raw Data (470-15'):
            green_columns.append(i)
        elif i.startswith(' Blank corrected based on Raw Data (570-15'):
            red_columns.append(i)
            
    od_df = df[od_columns].dropna()
    green_df = df[green_columns].dropna()
    red_df = df[red_columns].dropna()    
    
    return [od_df,green_df,red_df]

In [ ]:
'''plot_od_array

    This function takes the time in minutes list from the generate_time_in_minutes_list function, 
    an array of the values which should get visualized, a path as a string variable for saving the 
    figure and a header which serves as title for the figure.
    
    The function calculates the timepoint of the maximum slope of the data (point of highest growth rate).
    This is used for calculating the index of the exponential phase and the stationary phase, which are returned 
    by the function. Additionally it applies the savgol_filter to the data in arr. 
    
    :param header - Title for the figure (e.g., CPL_0025)
        :type str 
    :param time
        :type list[int]
    :param arr
        :type np.array
    :param save_path
        :type str
        
    :returns index_exponential_phase, index_stationary_phase
        :type tuple(int, int)

'''
def plot_od_array(header:str, time:list, arr:np.array, save_path:str):
    
    arr = savgol_filter(arr,window_length=19,polyorder=2)
    max_slope = [x - z for x, z in zip(list(arr[1:]), list(arr[:-1]))]
    
    for i,val in enumerate(max_slope):
        if val == max(max_slope[40:-40]):
            index = i
    
    for i,val in enumerate(arr):
        if val == max(arr):
            mindex = i
    
    index_exponential_phase = index
    index_stationary_phase = mindex
    
    max_val = arr[mindex] # str(round(max_val,2))
    
    mindex = time[mindex]
    index = time[index]
    
    vlabel = str(index) + " min."    
    vlabel2 = str(mindex) + " min."
    
    plt.figure(figsize=(12,8))
    plt.scatter(x=time,y=arr,color="blue", label="OD600 values")
    
    plt.vlines(index, 0, max(arr)+np.mean(arr), color="red", label="exponential phase")
    plt.vlines(mindex, 0, max(arr)+np.mean(arr), color="green", label="stationary phase")

    plt.ylim(0,max(arr)+0.25)
    plt.title(header)
    plt.grid()
    plt.xlabel("time in minutes", fontsize=15)
    plt.ylabel("OD600", fontsize=15)
    plt.tight_layout()
    plt.legend(fontsize=12)
    plt.savefig(save_path, dpi=300)
    plt.close()
    return index_exponential_phase, index_stationary_phase

In [ ]:
'''plot_red_vs_green
    
    This function takes the time in minutes list from the generate_time_in_minutes_list function, 
    an array of the values which should get visualized, an index for the exponential and the stationary phase,
    a path as a string variable for saving the figure and a header which serves as title for the figure.
    
    The function then uses the interp1d function of the scipy.interpolate module to interpolate the data with 
    a linear and a cubic method. All datapoints including the uninterpolated data are then visualized in a single figure
    with vlines of the exponential and stationary phases.
    
    :param header
        :type str
    :param time
        :type list
    :param red_vs_green
        :type np.array
    :param index_exp
        :type int
    :param index_stat
        :type int
    :param save_path
        :type str
    
    :returns none
        :tpye none

'''
def plot_red_vs_green(header:str, time:list, red_vs_green:np.array,index_exp:int, index_stat:int, save_path:str):
    f = interpolate.interp1d(time,red_vs_green , kind = 'linear')
    f2 = interpolate.interp1d(time,red_vs_green , kind = 'cubic')
    
    plt.figure(figsize=(12,8))
    plt.plot(time, red_vs_green, 'o', time, f(time), '-', time, f2(time), '--')
    plt.vlines(time[index_exp],0,max(red_vs_green)+np.mean(red_vs_green),color="red", label="RFP/GFP in peak exp. phase")
    plt.vlines(time[index_stat],0,max(red_vs_green)+np.mean(red_vs_green),color="green", label="RFP/GFP in stat. phase")

    plt.title(header)
    plt.ylim(0.1,max(red_vs_green)+0.1)
    plt.grid()
    plt.xlabel("time in minutes", fontsize=15)
    plt.ylabel("RFP/GFP Fluorescence", fontsize=15)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

In [ ]:
'''change_values
    
    This function is a helper function to change the "," signs to "." and it was used 
    to detect values that have been changed due to saving issues with Microsoft Excel. 
    
    :param header - column to parse
        :type str
    :param cont - CPL identifier in Content
        :type str
    :param df
        :type pd.DataFrame
    :param columns
        :type list[str]
    
    :returns arr
        :type np.array
'''
def change_values(header:str,cont:str,df:pd.DataFrame,columns:list)->np.array:
    arr = []
    for item in df[df[header] == cont].loc[:,columns].items():
        val = item[1].values[0]
        try:
            val = val.replace(",",".")
            val = float(val)
            arr.append(val)
        except Exception as e:
            if len(val) > 0:
                replacement = val[-1]
            else:
                replacement = 0.0
            print("[*] Date detected transform: {} to {}".format(val, replacement))
            val = replacement
            arr.append(val)
    return np.array(arr)

In [ ]:
'''plot_bar

    Function for drawing and saving a bar plot with values in l and a x-tick description in content.
    
    :param l
        :type list
    :param content
        :type str
    :param savep
        :type str
    :param title
        :type str

'''
def plot_bar(l:list, content:str, savep:str, title:str):
    plt.figure(figsize=(12,8))
    plt.bar(x=range(len(l)), height=l, linewidth=2, edgecolor='black')
    plt.xticks(range(len(content)), content, rotation=60)
    plt.ylim(0,max(l)+np.mean(l))
    plt.title(title)
    plt.tight_layout()
    plt.savefig(savep)
    plt.close()

In [ ]:
'''outlier_smoother
    
    Helper function to reduce the noise of data in an array.
    The function operates over a window of win values and tries to detect outlier values.
    Outlier values are defined by d > m*mdev where d is an array with all values - the median
    and mdev is the median of d. The key argument m defines the range which is allowed for values
    to deviate.
    
    :param x - data to "smooth"
        :type np.array
    :param time
        :type list[int]
    :param content - current CPL identifier
        :type str
    :param index_exp
        :type int
    :param index_stat
        :type int
    :param savep
        :type str
    
    :returns none
        :type none
'''
def outlier_smoother(x:np.array, time:list, content:str, index_exp:int, index_stat:int, savep:str, m=3, win=3, plots=False):
    ''' finds outliers in x, points > m*mdev(x) [mdev:median deviation] 
    and replaces them with the median of win points around them '''
    x_corr = np.copy(x)
    xtick_locs = np.arange(0,len(time),step=20)# range(len(time))
    xtick_labels = time[::20]
    d = np.abs(x - np.median(x))
    mdev = np.median(d)
    idxs_outliers = np.nonzero(d > m*mdev)[0]
    for i in idxs_outliers:
        if i-win < 0:
            x_corr[i] = np.median(np.append(x[0:i], x[i+1:i+win+1]))
        elif i+win+1 > len(x):
            x_corr[i] = np.median(np.append(x[i-win:i], x[i+1:len(x)]))
        else:
            x_corr[i] = np.median(np.append(x[i-win:i], x[i+1:i+win+1]))
    if plots:
        plt.figure('outlier_smoother '+content, clear=True)
        plt.plot(x, label='orig.', lw=5)
        plt.plot(idxs_outliers, x[idxs_outliers], 'ro', label='outliers')                                                                                                                    
        plt.plot(x_corr, '-o', label='corrected')
        
        # plt.vlines(time[index_exp],0,max(x),color="red")
        # plt.vlines(time[index_stat],0,max(x),color="green")
        
        plt.legend()
        plt.xticks(xtick_locs,xtick_labels)
        plt.xlabel("time in minutes")
        plt.tight_layout()
        plt.savefig(savep)
        plt.close()
    
    return x_corr

In [ ]:
'''plot_phase_bar_plot

    This function can be used to reproduce result figure 4a of the manuscript. As input it takes a
    list of CPL identifiers, the normalized RFU values as described in equation 2 of the manuscript 
    for the stationary and for the exponential growth phases and some additional settings for 
    customizing the horizontal bar chart produced by this function.
    
    :param content - list of CPL identifer
        :type list[str]
    :param stat_phase - list of RFP/GFP values of the stationary phase
        :type list
    :param exp_phase - list of RFP/GFP values of the exponential phase
        :type list
    :param savep - filename and path for the figure
        :type str
    
    :returns none
        :type none
'''
def plot_phase_bar_plot(content:list,stat_phase:list,exp_phase:list,savep:str, xlim=1, legend_position="best", fsize=15, fsize_text=8, fig_size=(14,10)):

    try:
        if len(content) == len(stat_phase) == len(exp_phase):
            width = 0.4
            ind = np.arange(len(content))
            
            plt.figure(figsize=fig_size)
            bar1 = plt.barh(ind, exp_phase, width, label="RFP/GFP exponential phase", color="blue", edgecolor="black")
            bar2 = plt.barh(ind + width, stat_phase, width, label="RFP/GFP stationary phase", color="orange", edgecolor="black")
            
            for rect in bar1 + bar2:
                if rect.get_width() > 0:
                    if rect.get_width() > 0.0099999999:
                        if rect.get_width() > 0.1:
                            plt.text(rect.get_width() + 0.012, rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)
                        else:
                            plt.text(rect.get_width() + 0.015, rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)

                    else:
                        plt.text(rect.get_width() + 0.015, rect.get_y() - 0.05, f'{np.abs(rect.get_width()):.3f}', ha='center', va='bottom', fontsize=fsize_text)

            plt.xlabel("RFP/GFP fluorescence values", fontsize=fsize)
            plt.ylabel("Curvibacter Promoter Library (CPL) identifier", fontsize=fsize)
            # plt.title("Comparison of CPL-promoter constructs in exponentially and stationary growth phases")
            plt.yticks(ind + width / 2, content, fontsize=fsize-2)
            plt.xticks(fontsize=12)
            plt.legend(loc=legend_position, fontsize=fsize-2)
            plt.tight_layout()
            plt.xlim(0,xlim)
            plt.grid(axis="x")
            plt.savefig(savep)
            plt.close()
        else:
            raise Exception("[-] lists do not have the same length: content: {} stat_phase: {}  exp_phase: {}"
                            .format(len(content),len(stat_phase),len(exp_phase)))
    except Exception as e:
        raise Exception("[-] ERROR plotting phase bar plot with exception: {}".format(e))

In [ ]:
'''plot_min_max_bar_plot
    
    This function is similar to the plot_phase_bar_plot except for plotting 
    the maximum and minimum values of the RFP/GFP values.
    
    :param content
        :type list[str]
    :param min_values
        :type list[float]
    :param max_values
        :type list[float]
    :param savep
        :type str
    
    :returns none
        :type none
'''
def plot_min_max_bar_plot(content:list,min_values:list,max_values:list,savep:str):

    try:
        if len(content) == len(min_values) == len(max_values):
            width = 0.4
            ind = np.arange(len(content))
            
            plt.figure(figsize=(14,10))
            bar1 = plt.barh(ind, min_values, width, label="RFP/GFP minimal values", color="blue", edgecolor="black")
            bar2 = plt.barh(ind + width, max_values, width, label="RFP/GFP maximal values", color="orange", edgecolor="black")
            
            for rect in bar1 + bar2:
                if rect.get_width() > 0:
                    if rect.get_width() > 0.0099999999:
                        plt.text(rect.get_width() + 0.015, rect.get_y() , f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=8)
                    else:
                        plt.text(rect.get_width() + 0.015, rect.get_y() , f'{np.abs(rect.get_width()):.3f}', ha='center', va='bottom', fontsize=8)

            plt.xlabel("RFP/GFP fluorescence values")
            plt.ylabel("Curvibacter Promoter Library (CPL) identifier")
            #plt.title("Comparison of CPL-promoter constructs in exponentially and stationary growth phases")
            plt.yticks(ind + width / 2, content)
            plt.legend(loc="best")
            plt.tight_layout()
            plt.xlim(0,1)
            plt.grid()
            plt.savefig(savep)
            plt.close()
        else:
            raise Exception("[-] lists do not have the same length: content: {} stat_phase: {}  exp_phase: {}"
                            .format(len(content),len(stat_phase),len(exp_phase)))
    except Exception as e:
        raise Exception("[-] ERROR plotting phase bar plot with exception: {}".format(e))

In [ ]:
'''plot_content
    
    Wrapper function for plotting OD and RFU data. 
    This function combines the above described functions.

'''
def plot_content(df:pd.DataFrame, save_path:str):
   
    time = generate_time_in_minutes_list(df)
    datasets = extract_datasets(df)
    content = extract_content(df)
    
    min_val = []
    max_val = []
    
    exp_phase = []
    stat_phase = []
    
    real_cont = []
    for cont in content:
        od_arr = change_values("Content",cont,datasets[0], datasets[0].columns[2:])
        green_arr = change_values("Content",cont,datasets[1], datasets[1].columns[2:])
        red_arr = change_values("Content",cont,datasets[2], datasets[2].columns[2:])
                
        savep = save_path + cont + "_OD600.png"
        index_exp, index_stat = plot_od_array(cont, time, od_arr, savep)
        savep = save_path + cont + "_red_vs_green_sm.png"
        red_vs_green = np.array(red_arr.astype(float) / green_arr.astype(float))
        smoothed_red_vs_green = outlier_smoother(red_vs_green,time, cont, index_exp, index_stat, savep,plots=True)
        savep = save_path + cont + "_red_vs_green_sm_outlier_removed.png"
        plot_red_vs_green(cont,time, smoothed_red_vs_green,index_exp, index_stat, savep)
        
        min_val.append(min(smoothed_red_vs_green))
        max_val.append(max(smoothed_red_vs_green))
        exp_phase.append(red_vs_green[index_exp])
        stat_phase.append(red_vs_green[index_stat])
        real_cont.append(cont)
        
        od_color = "black"
        rfp_gfp_color = "darkorange"
        gfp_color = "green"
        rfp_color = "red"
        vline_color = "red"
        savep = save_path + "combined_od_red_vs_green_" + cont + ".png"
        plot_combined_od_red_vs_green_plot(df,cont,od_color,rfp_gfp_color,vline_color, savep)
        
    savep = save_path + "local_maxima.png"
    plot_bar(max_val, content, savep, "Local maxima of RFP/GFP")
    savep = save_path + "local_minima.png"
    plot_bar(min_val, content, savep, "Local minima of RFP/GFP")
    savep = save_path + "exponential_phase.png"
    plot_bar(exp_phase, content, savep, "Values of RFP/GFP in exponential phase")
    savep = save_path + "stationary_phase.png"
    plot_bar(stat_phase, content, savep, "Values of RFP/GFP in stationary phase")
    savep = save_path + "exponential_stationary_phase_bar_plots.png"
    plot_phase_bar_plot(content, stat_phase, exp_phase, savep)
    savep = save_path + "min_max_bar_plots.png"
    plot_min_max_bar_plot(content, min_values=min_val, max_values=max_val, savep=savep)
    dataframe = pd.DataFrame(zip(content, exp_phase, stat_phase, min_val, max_val),
                         columns=["CPL","EXP_PHASE","STAT_PHASE","MIN_VAL","MAX_VAL"])
    dataframe.to_csv(save_path+"comorehensive_dataframe.csv")
    return content, min_val, max_val, exp_phase, stat_phase
        

In [ ]:
'''collect_interesting_cpl_data

    This function replaces the "_" sign in the CPL identifier with " " and extracts
    all CPL identifier in the current dataframe to fill the dictionary all_cpls with CPL identifier
    as keys and RFP/GFP values of the exp and stat phases as values. Only CPLs in the list 
    cpls_to_keep are processed.
    
    :param cpls_to_keep
        :type list[str]
    :param content
        :type list[str]
    :param exp_phase
        :type list[float]
    :param stat_phase
        :type list[float]
    :param all_cpls
        :type dict
        
    :returns all_cpls
        :type dict

'''
def collect_interesting_cpl_data(cpls_to_keep:list,content:list,exp_phase:list,stat_phase:list,all_cpls:dict)->dict:
    content = [cpl.replace("_","") for cpl in content]
    
    for idx,cpl in enumerate(content):
        if cpl in cpls_to_keep:
            if cpl in list(all_cpls.keys()):
                all_cpls[cpl].append(exp_phase[idx])
                all_cpls[cpl].append(stat_phase[idx])
            else:
                all_cpls[cpl] = [exp_phase[idx], stat_phase[idx]]
    return all_cpls

In [ ]:
'''clean_collected_dictionary

    Helper function to return lists with values of the all_cpls dictionary.
    The promoter sequence of the J23100 normalization promoter construct is not 
    returned.
    
    :param all_cpls
        :type dict
    
    :returns [content, exp_phase, stat_phase]
        :type list

'''
def clean_collected_dictionary(all_cpls:dict)->list:
    content = []
    exp_phase = []
    stat_phase = []
    for key in all_cpls.keys():
        if key != "CPL0017":
            content.append(key)
            exp_phase.append(all_cpls[key][0])
            stat_phase.append(all_cpls[key][1])
            
        elif key == "CPL0017":
            content.append(key)
            exp_phase.append((all_cpls[key][0]+all_cpls[key][2])/2)
            stat_phase.append((all_cpls[key][1]+all_cpls[key][3])/2)
        else:
            print("ERROR")
    return [content,exp_phase,stat_phase]

# R2A Dataset

In [ ]:
# restrict analysis to interesting CPLs
cpls = "0025,0022,0021,0017,0012,0008,0006,0005,0003,0002,0001,0017,0114,0112,0109,0106,0095,0083,0077,0075,0072,0071,0070,0069,0067,0065"
cpls = cpls.split(",")
cpls = ["CPL"+str(cpl) for cpl in cpls]

# this dictionary will inherit all informations from replicates 3 of cohort 1 and 2
all_cpls = {}

In [ ]:
# reading dataframes of repetition one
df1 = pd.read_csv(data_path + "r2aplus_30degree/csv/cohort1rep1_reconstituted.csv", delimiter="\t")
df2 = pd.read_csv(data_path + "r2aplus_30degree/csv/co1rep2.csv",skiprows=11, delimiter=";")
df3 = pd.read_csv(data_path + "r2aplus_30degree/csv/co1rep3.csv", skiprows=11, delimiter=";")

In [ ]:
# defining colors for the plots
od_color = "black"
rfp_gfp_color = "darkorange"
gfp_color = "green"
rfp_color = "red"
vline_color = "red"

In [ ]:
'''plot_combined_od_gfp

    This function plots a combination of the measured OD600 and GFP values to demonstrate
    the correlation between GFP and OD600 values.
    
    :param dataframe
        :type pd.DataFrame
    :param cpl_number
        :type str
    :param od_color
        :type str
    :pram rfp_gfp_color - in fact here it is just GFP
        :type str
    :param vline_color
        :type str
    :param savep
        :type str

'''
def plot_combined_od_gfp(dataframe:pd.DataFrame,cpl_number:str,od_color:str,rfp_gfp_color:str,vline_color:str,savep:str)->int:
    
    time = generate_time_in_minutes_list(dataframe)
    
    datasets = extract_datasets(dataframe)
    content = extract_content(dataframe)
    
    od_arr = change_values("Content",cpl_number,datasets[0], datasets[0].columns[2:])
    green_arr = change_values("Content",cpl_number,datasets[1], datasets[1].columns[2:])
    red_arr = change_values("Content",cpl_number,datasets[2], datasets[2].columns[2:])
    od_data = savgol_filter(od_arr,window_length=19,polyorder=2)
    gfp_data = savgol_filter(green_arr,window_length=19,polyorder=2)

    max_slope = [x - z for x, z in zip(list(od_data[1:]), list(od_data[:-1]))]

    for i,val in enumerate(max_slope):
        if val == max(max_slope[40:-40]):
            index = i

    for i,val in enumerate(od_data):
        if val == max(od_data):
            mindex = i

    index_exponential_phase = index
    index_stationary_phase = mindex

    max_val = od_data[mindex]
    mindex = time[mindex]
    index = time[index]
    vlabel = str(index) + " min."    

    fig, ax = plt.subplots(figsize=(12,8))
    axes = [ax, ax.twinx()]

    colors = ('orange','blue')

    axes[0].plot(time,od_data,linestyle="none", marker='o', color=od_color)
    axes[0].set_ylabel("OD600", color=od_color, fontsize=15)
    axes[0].tick_params(axis='y', colors=od_color, labelsize=12)

    axes[1].plot(time,gfp_data,linestyle='none', marker='o', color=rfp_gfp_color)
    axes[1].set_ylabel("RFU GFP", color=rfp_gfp_color, fontsize=15)
    axes[1].tick_params(axis='y', colors=rfp_gfp_color, labelsize=12)
    axes[1].tick_params(axis='x', colors="black", labelsize=12)


    axes[0].set_xlabel("time in minutes", fontsize=15)
    ax.grid()

    ax.vlines(index, 0, max(od_data)+np.mean(od_data), linestyles="--", color=vline_color, label="exponential phase")
    ax.vlines(mindex, 0, max(od_data)+np.mean(od_data), linestyles="-", color=vline_color, label="stationary phase")

    ax.legend(loc="lower right", fontsize=15)

    plt.savefig(savep, dpi=400)
    plt.close()
    return 0

In [ ]:
savep = "../results/cpl_0022_gfp_od600_correlation.png"
cpl_number = "CPL_0022"
plot_combined_od_gfp(df3, cpl_number, od_color, gfp_color, vline_color, savep)

In [ ]:
savep = "../results/cpl_0017_gfp_od600_correlation.png"
cpl_number = "CPL_0017"
plot_combined_od_gfp(df3, cpl_number, od_color, gfp_color, vline_color, savep)

In [ ]:
'''plot_combined_od_red_vs_green_plot

    This function plots a figure with a combination of OD600 and RFP/GFP
    values.
    
    :param dataframe
        :type pd.DataFrame
    :param cpl_number
        :type str
    :param od_color
        :type str
    :param rfp_gfp_color
        :type str
    :param vline_color
        :type str
    :param savep
        :type str
    
    :returns returncode
        :type int

'''
def plot_combined_od_red_vs_green_plot(dataframe:pd.DataFrame,cpl_number:str,od_color:str,
                                       rfp_gfp_color:str,vline_color:str,savep:str)->int:
    time = generate_time_in_minutes_list(dataframe)
    datasets = extract_datasets(dataframe)
    content = extract_content(dataframe)

    od_arr = change_values("Content",cpl_number,datasets[0], datasets[0].columns[2:])
    green_arr = change_values("Content",cpl_number,datasets[1], datasets[1].columns[2:])
    red_arr = change_values("Content",cpl_number,datasets[2], datasets[2].columns[2:])
    red_vs_green = np.array(red_arr.astype(float) / green_arr.astype(float))
    od_data = savgol_filter(od_arr,window_length=19,polyorder=2)
    max_slope = [x - z for x, z in zip(list(od_data[1:]), list(od_data[:-1]))]

    for i,val in enumerate(max_slope):
        if val == max(max_slope[40:-40]):
            index = i

    for i,val in enumerate(od_data):
        if val == max(od_data):
            mindex = i

    index_exponential_phase = index
    index_stationary_phase = mindex

    max_val = od_data[mindex]
    
    index_exponential_phase = index
    index_stationary_phase = mindex
    
    mindex = time[mindex]
    index = time[index]
    vlabel = str(index) + " min."    


    # smoothing the data with a window of 3 and a range of 3
    red_vs_green = outlier_smoother(red_vs_green,time, cpl_number, index_exponential_phase, index_stationary_phase, savep,plots=False)

    fig, ax = plt.subplots(figsize=(12,8))

    axes = [ax, ax.twinx()]
    axes[0].plot(time,od_data,linestyle="none", marker='o', color=od_color)
    axes[0].set_ylabel("OD600", color=od_color, fontsize=17)
    axes[0].tick_params(axis='y', colors=od_color, labelsize=17)
    axes[0].tick_params(axis='x', colors="black", labelsize=17)

    axes[1].plot(time,red_vs_green,linestyle='-',marker="o", color=rfp_gfp_color)
    axes[1].set_ylabel("RFP/GFP", color=rfp_gfp_color, fontsize=17)
    axes[1].tick_params(axis='y', colors=rfp_gfp_color, labelsize=17)
    axes[1].tick_params(axis='x', colors="black", labelsize=17)

    axes[1].set_ylim(0,0.8)

    axes[0].set_xlabel("time in minutes", fontsize=15)
    axes[0].grid(axis="y",color="black", linestyle="--", alpha=0.5)
    axes[1].grid(axis="y",color=rfp_gfp_color,linestyle="--", alpha=0.5)
    
    ax.vlines(index, 0, max(od_data)+np.mean(od_data), linestyles="--", color=vline_color, label="exponential phase")
    ax.vlines(mindex, 0, max(od_data)+np.mean(od_data), linestyles="-", color=vline_color, label="stationary phase")

    ax.legend(loc="lower right", fontsize=15)

    plt.savefig(savep, dpi=400)
    plt.close()
    return 0

In [ ]:
# subplots with CPL_0022 / CPL_0025 / CPL_0095 / CPL_0017
'''get_data_for_subplot

    This function is used for the creation of Figure 6 of the Manuscript.
    It returns the OD and RFU values for a specific CPL identifier (cpl_number).
    The index is the timepoint of the maximum slope within the exponential growth phase.
    The mindex is the timepoint of the maximum value of the OD600 dataframe.
    
    :param dataframe
        :type pd.DataFrane
    
    :returns [index, mindex, od_data, red_vs_green, time]
        :type list[int, int, np.array, np.array, list[int]]

'''
def get_data_for_subplot(dataframe:pd.DataFrame, cpl_number:str)->list:
    time = generate_time_in_minutes_list(dataframe)
    datasets = extract_datasets(dataframe)
    content = extract_content(dataframe)

    od_arr = change_values("Content",cpl_number,datasets[0], datasets[0].columns[2:])
    green_arr = change_values("Content",cpl_number,datasets[1], datasets[1].columns[2:])
    red_arr = change_values("Content",cpl_number,datasets[2], datasets[2].columns[2:])
    red_vs_green = np.array(red_arr.astype(float) / green_arr.astype(float))
    od_data = savgol_filter(od_arr,window_length=19,polyorder=2)
    max_slope = [x - z for x, z in zip(list(od_data[1:]), list(od_data[:-1]))]

    for i,val in enumerate(max_slope):
        if val == max(max_slope[40:-40]):
            index = i

    for i,val in enumerate(od_data):
        if val == max(od_data):
            mindex = i

    index_exponential_phase = index
    index_stationary_phase = mindex

    max_val = od_data[mindex]

    index_exponential_phase = index
    index_stationary_phase = mindex

    mindex = time[mindex]
    index = time[index]
    vlabel = str(index) + " min."    
    
    savep = "none"
    red_vs_green = outlier_smoother(red_vs_green,time, cpl_number, index_exponential_phase, index_stationary_phase, savep, plots=False)
    return [index, mindex, od_data, red_vs_green, time]

In [ ]:
#df_cpl_0095 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep3.csv", skiprows=11, delimiter=";")
df_cpl_0095 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep2.csv",skiprows=11, delimiter=";")

In [ ]:
index_cpl_0022, mindex_cpl_0022, od_data_cpl_0022, red_vs_green_cpl_0022, time_cpl_0022 = get_data_for_subplot(df3, "CPL_0022")
index_cpl_0025, mindex_cpl_0025, od_data_cpl_0025, red_vs_green_cpl_0025, time_cpl_0025 = get_data_for_subplot(df3, "CPL_0025")
index_cpl_0017, mindex_cpl_0017, od_data_cpl_0017, red_vs_green_cpl_0017, time_cpl_0017 = get_data_for_subplot(df3, "CPL_0017")
index_cpl_0095, mindex_cpl_0095, od_data_cpl_0095, red_vs_green_cpl_0095, time_cpl_0095 = get_data_for_subplot(df_cpl_0095, "CPL_0095")


In [ ]:
plot_list = [[index_cpl_0017, mindex_cpl_0017, od_data_cpl_0017, red_vs_green_cpl_0017, time_cpl_0017, "CPL0017"],
           [index_cpl_0025, mindex_cpl_0025, od_data_cpl_0025, red_vs_green_cpl_0025, time_cpl_0025, "CPL0025"],
           [index_cpl_0022, mindex_cpl_0022, od_data_cpl_0022, red_vs_green_cpl_0022, time_cpl_0022, "CPL0022"],
           [index_cpl_0095, mindex_cpl_0095, od_data_cpl_0095, red_vs_green_cpl_0095, time_cpl_0095, "CPL0095"]]

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(figsize=(15,10), nrows=2, ncols=2)

axes = [ (ax1, ax1.twinx()), (ax2, ax2.twinx()), (ax3, ax3.twinx()), (ax4, ax4.twinx())]

for index, data in enumerate(plot_list):
    axes[index][0].plot(data[4],data[2],linestyle="none", marker='o', color=od_color)
    
    if index != 1 and index != 3:
        #axes[index][0].set_ylabel("OD600", color=od_color, fontsize=17)
        axes[index][0].tick_params(axis='y', colors=od_color, labelsize=17)
        axes[index][0].tick_params(axis='x', colors="black", labelsize=17)
    else:
        axes[index][0].tick_params(left = False, right = False , labelleft = False) 
        axes[index][0].tick_params(axis='x', colors="black", labelsize=17)
        
    axes[index][1].plot(data[4],data[3],linestyle='-',marker="o", color=rfp_gfp_color)
    
    if index != 0 and index != 2:
        #axes[index][1].set_ylabel("RFP/GFP", color=rfp_gfp_color, fontsize=17)
        axes[index][1].tick_params(axis='y', colors=rfp_gfp_color, labelsize=17)
        axes[index][1].tick_params(axis='x', colors="black", labelsize=17)
    else:
        axes[index][1].tick_params(left = False, right = False , labelleft = False, labelright=False) 
    
    axes[index][1].set_ylim(0,0.8)
    axes[index][0].set_ylim(0,2.0)
    
        
    axes[index][0].grid(axis="y",color="black", linestyle="--", alpha=0.5)
    axes[index][1].grid(axis="y",color=rfp_gfp_color,linestyle="--", alpha=0.5)

    axes[index][1].vlines(data[0], 0, max(data[2])+np.mean(data[2]), linestyles="--", color=vline_color, label="exponential phase")
    axes[index][1].vlines(data[1], 0, max(data[2])+np.mean(data[2]), linestyles="-", color=vline_color, label="stationary phase")
    
    if index == 3:
        axes[index][1].legend(loc="upper left", fontsize=15)
    
    axes[index][0].set_title(data[5], fontsize=17)
    
#fig.supxlabel('time in minutes', fontsize=20)
#fig.supylabel('OD600', fontsize=20, x=-0.02)
fig.text(x=-0.03, y=0.5, s="OD600", size=25, fontweight='bold', rotation=90, ha='center', va='center')
fig.text(x=0.55, y=-0.01, s="time in minutes", size=25, fontweight='bold', rotation=0, ha='center', va='center')

fig.text(x=1.15, y=0.5, s="RFP/GFP", size=25, fontweight='bold', rotation=-90, ha='center', va='center', color="orange")
fig.text(x=0.58, y= + 1.03, s="Fluorescence Activity of Individual Candidate Sequences ", size=25, fontweight='bold', rotation=0, ha='center', va='center')

plt.tight_layout()
plt.subplots_adjust(right=1.1,)
plt.savefig("../results/subplots_cpls.png", dpi=400,  bbox_inches = 'tight')

# Mean Values For Combined Subplots

In [ ]:
index_cpl_0022_1, mindex_cpl_0022_1, od_data_cpl_0022_1, red_vs_green_cpl_0022_1, time_cpl_0022_1 = get_data_for_subplot(df1, "CPL0022")
index_cpl_0022_2, mindex_cpl_0022_2, od_data_cpl_0022_2, red_vs_green_cpl_0022_2, time_cpl_0022_2 = get_data_for_subplot(df2, "CPL_0022")
index_cpl_0022_3, mindex_cpl_0022_3, od_data_cpl_0022_3, red_vs_green_cpl_0022_3, time_cpl_0022_3 = get_data_for_subplot(df3, "CPL_0022")

index_cpl_0022 = (index_cpl_0022_1 + index_cpl_0022_2 + index_cpl_0022_3)/3
mindex_cpl_0022 = (mindex_cpl_0022_1 + mindex_cpl_0022_2 + mindex_cpl_0022_3)/3

min_measurement_index = min([len(od_data_cpl_0022_1),len(od_data_cpl_0022_2),len(od_data_cpl_0022_3)])
od_data_cpl_0022 = od_data_cpl_0022_1[:min_measurement_index] + od_data_cpl_0022_2[:min_measurement_index] + od_data_cpl_0022_3[:min_measurement_index]
od_data_cpl_0022 = od_data_cpl_0022/3

red_vs_green_cpl_0022 = red_vs_green_cpl_0022_1[:min_measurement_index] + red_vs_green_cpl_0022_2[:min_measurement_index] + red_vs_green_cpl_0022_3[:min_measurement_index]
red_vs_green_cpl_0022 = red_vs_green_cpl_0022/3

time_cpl_0022 = np.array(time_cpl_0022_1[:min_measurement_index]) + np.array(time_cpl_0022_2[:min_measurement_index]) + np.array(time_cpl_0022_3[:min_measurement_index])
time_cpl_0022 = time_cpl_0022/3

In [ ]:
index_cpl_0025_1, mindex_cpl_0025_1, od_data_cpl_0025_1, red_vs_green_cpl_0025_1, time_cpl_0025_1 = get_data_for_subplot(df1, "CPL0025")
index_cpl_0025_2, mindex_cpl_0025_2, od_data_cpl_0025_2, red_vs_green_cpl_0025_2, time_cpl_0025_2 = get_data_for_subplot(df2, "CPL_0025")
index_cpl_0025_3, mindex_cpl_0025_3, od_data_cpl_0025_3, red_vs_green_cpl_0025_3, time_cpl_0025_3 = get_data_for_subplot(df3, "CPL_0025")

index_cpl_0025 = (index_cpl_0025_1 + index_cpl_0025_2 + index_cpl_0025_3)/3
mindex_cpl_0025 = (mindex_cpl_0025_1 + mindex_cpl_0025_2 + mindex_cpl_0025_3)/3

min_measurement_index = min([len(od_data_cpl_0025_1),len(od_data_cpl_0025_2),len(od_data_cpl_0025_3)])
od_data_cpl_0025 = od_data_cpl_0025_1[:min_measurement_index] + od_data_cpl_0025_2[:min_measurement_index] + od_data_cpl_0025_3[:min_measurement_index]
od_data_cpl_0025 = od_data_cpl_0025/3

red_vs_green_cpl_0025 = red_vs_green_cpl_0025_1[:min_measurement_index] + red_vs_green_cpl_0025_2[:min_measurement_index] + red_vs_green_cpl_0025_3[:min_measurement_index]
red_vs_green_cpl_0025 = red_vs_green_cpl_0025/3

time_cpl_0025 = np.array(time_cpl_0025_1[:min_measurement_index]) + np.array(time_cpl_0025_2[:min_measurement_index]) + np.array(time_cpl_0025_3[:min_measurement_index])
time_cpl_0025 = time_cpl_0025/3

In [ ]:
df3_cpl_0095 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep3.csv", skiprows=11, delimiter=";")
df2_cpl_0095 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep2.csv",skiprows=11, delimiter=";")
df1_cpl_0095 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep1.csv",skiprows=11, delimiter=";")

In [ ]:
index_cpl_0095_1, mindex_cpl_0095_1, od_data_cpl_0095_1, red_vs_green_cpl_0095_1, time_cpl_0095_1 = get_data_for_subplot(df1_cpl_0095, "CPL_0095")
index_cpl_0095_2, mindex_cpl_0095_2, od_data_cpl_0095_2, red_vs_green_cpl_0095_2, time_cpl_0095_2 = get_data_for_subplot(df2_cpl_0095, "CPL_0095")
index_cpl_0095_3, mindex_cpl_0095_3, od_data_cpl_0095_3, red_vs_green_cpl_0095_3, time_cpl_0095_3 = get_data_for_subplot(df3_cpl_0095, "CPL_0095")

index_cpl_0095 = (index_cpl_0095_1 + index_cpl_0095_2 + index_cpl_0095_3)/3
mindex_cpl_0095 = (mindex_cpl_0095_1 + mindex_cpl_0095_2 + mindex_cpl_0095_3)/3

min_measurement_index = min([len(od_data_cpl_0095_1),len(od_data_cpl_0095_2),len(od_data_cpl_0095_3)])
od_data_cpl_0095 = od_data_cpl_0095_1[:min_measurement_index] + od_data_cpl_0095_2[:min_measurement_index] + od_data_cpl_0095_3[:min_measurement_index]
od_data_cpl_0095 = od_data_cpl_0095/3

red_vs_green_cpl_0095 = red_vs_green_cpl_0095_1[:min_measurement_index] + red_vs_green_cpl_0095_2[:min_measurement_index] + red_vs_green_cpl_0095_3[:min_measurement_index]
red_vs_green_cpl_0095 = red_vs_green_cpl_0095/3

time_cpl_0095 = np.array(time_cpl_0095_1[:min_measurement_index]) + np.array(time_cpl_0095_2[:min_measurement_index]) + np.array(time_cpl_0095_3[:min_measurement_index])
time_cpl_0095 = time_cpl_0095/3

In [ ]:
index_cpl_0017_1, mindex_cpl_0017_1, od_data_cpl_0017_1, red_vs_green_cpl_0017_1, time_cpl_0017_1 = get_data_for_subplot(df1_cpl_0095, "CPL_0017")
index_cpl_0017_2, mindex_cpl_0017_2, od_data_cpl_0017_2, red_vs_green_cpl_0017_2, time_cpl_0017_2 = get_data_for_subplot(df2_cpl_0095, "CPL_0017")
index_cpl_0017_3, mindex_cpl_0017_3, od_data_cpl_0017_3, red_vs_green_cpl_0017_3, time_cpl_0017_3 = get_data_for_subplot(df3_cpl_0095, "CPL_0017")

index_cpl_0017 = (index_cpl_0017_1 + index_cpl_0017_2 + index_cpl_0017_3)/3
mindex_cpl_0017 = (mindex_cpl_0017_1 + mindex_cpl_0017_2 + mindex_cpl_0017_3)/3

min_measurement_index = min([len(od_data_cpl_0017_1),len(od_data_cpl_0017_2),len(od_data_cpl_0017_3)])
od_data_cpl_0017 = od_data_cpl_0017_1[:min_measurement_index] + od_data_cpl_0017_2[:min_measurement_index] + od_data_cpl_0017_3[:min_measurement_index]
od_data_cpl_0017 = od_data_cpl_0017/3

red_vs_green_cpl_0017 = red_vs_green_cpl_0017_1[:min_measurement_index] + red_vs_green_cpl_0017_2[:min_measurement_index] + red_vs_green_cpl_0017_3[:min_measurement_index]
red_vs_green_cpl_0017 = red_vs_green_cpl_0017/3

time_cpl_0017 = np.array(time_cpl_0017_1[:min_measurement_index]) + np.array(time_cpl_0017_2[:min_measurement_index]) + np.array(time_cpl_0017_3[:min_measurement_index])
time_cpl_0017 = time_cpl_0017/3

In [ ]:
plot_list = [[index_cpl_0017, mindex_cpl_0017, od_data_cpl_0017, red_vs_green_cpl_0017, time_cpl_0017, "CPL0017"],
           [index_cpl_0025, mindex_cpl_0025, od_data_cpl_0025, red_vs_green_cpl_0025, time_cpl_0025, "CPL0025"],
           [index_cpl_0022, mindex_cpl_0022, od_data_cpl_0022, red_vs_green_cpl_0022, time_cpl_0022, "CPL0022"],
           [index_cpl_0095, mindex_cpl_0095, od_data_cpl_0095, red_vs_green_cpl_0095, time_cpl_0095, "CPL0095"]]

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(figsize=(15,10), nrows=2, ncols=2)

axes = [ (ax1, ax1.twinx()), (ax2, ax2.twinx()), (ax3, ax3.twinx()), (ax4, ax4.twinx())]

for index, data in enumerate(plot_list):
    axes[index][0].plot(data[4],data[2],linestyle="none", marker='o', color=od_color)
    
    if index != 1 and index != 3:
        #axes[index][0].set_ylabel("OD600", color=od_color, fontsize=17)
        axes[index][0].tick_params(axis='y', colors=od_color, labelsize=17)
        axes[index][0].tick_params(axis='x', colors="black", labelsize=17)
    else:
        axes[index][0].tick_params(left = False, right = False , labelleft = False) 
        axes[index][0].tick_params(axis='x', colors="black", labelsize=17)
        
    axes[index][1].plot(data[4],data[3],linestyle='-',marker="o", color=rfp_gfp_color)
    
    if index != 0 and index != 2:
        #axes[index][1].set_ylabel("RFP/GFP", color=rfp_gfp_color, fontsize=17)
        axes[index][1].tick_params(axis='y', colors=rfp_gfp_color, labelsize=17)
        axes[index][1].tick_params(axis='x', colors="black", labelsize=17)
    else:
        axes[index][1].tick_params(left = False, right = False , labelleft = False, labelright=False) 
    
    axes[index][1].set_ylim(0,0.8)
    axes[index][0].set_ylim(0,2.0)
    
        
    axes[index][0].grid(axis="y",color="black", linestyle="--", alpha=0.5)
    axes[index][1].grid(axis="y",color=rfp_gfp_color,linestyle="--", alpha=0.5)

    axes[index][1].vlines(data[0], 0, max(data[2])+np.mean(data[2]), linestyles="--", color=vline_color, label="exponential phase")
    axes[index][1].vlines(data[1], 0, max(data[2])+np.mean(data[2]), linestyles="-", color=vline_color, label="stationary phase")
    
    if index == 3:
        axes[index][1].legend(loc="upper left", fontsize=15)
    
    axes[index][0].set_title(data[5], fontsize=17)
    
#fig.supxlabel('time in minutes', fontsize=20)
#fig.supylabel('OD600', fontsize=20, x=-0.02)
fig.text(x=-0.03, y=0.5, s="OD600", size=25, fontweight='bold', rotation=90, ha='center', va='center')
fig.text(x=0.55, y=-0.01, s="time in minutes", size=25, fontweight='bold', rotation=0, ha='center', va='center')

fig.text(x=1.15, y=0.5, s="RFP/GFP", size=25, fontweight='bold', rotation=-90, ha='center', va='center', color="orange")
fig.text(x=0.58, y= + 1.03, s="Fluorescence Activity of Individual Candidate Sequences ", size=25, fontweight='bold', rotation=0, ha='center', va='center')

plt.tight_layout()
plt.subplots_adjust(right=1.1,)
plt.savefig("../results/subplots_cpls_mean_values.png", dpi=400,  bbox_inches = 'tight')

# Solo Plots And Data Aquisition for Figure 4a

In [ ]:
savep="../results/CPL_0022_combined_plot.png"
cpl_number = "CPL_0022"
red_vs_green = plot_combined_od_red_vs_green_plot(df3,"CPL_0022",od_color,rfp_gfp_color,vline_color, savep)

In [ ]:
savep="../results/CPL_0025_combined_plot.png"
cpl_number = "CPL_0025"
red_vs_green = plot_combined_od_red_vs_green_plot(df3,"CPL_0025",od_color,rfp_gfp_color,vline_color, savep)

In [ ]:
savep="../results/CPL_0017_combined_plot.png"
cpl_number = "CPL_0017"
plot_combined_od_red_vs_green_plot(df3,"CPL_0017",od_color,rfp_gfp_color,vline_color, savep)

In [ ]:
save_path = result_path + "r2aplus_30degree_cohorte1/replicates1/"
if isdir(result_path + "r2aplus_30degree_cohorte1/") == False:
    mkdir(result_path + "r2aplus_30degree_cohorte1/")
if isdir(save_path) == False:
    mkdir(save_path)
content_df1, min_val, max_val, exp_phase_df1, stat_phase_df1 = plot_content(df1, save_path)

all_cpls_df1 = {}
all_cpls_df1 = collect_interesting_cpl_data(cpls,content_df1,exp_phase_df1,stat_phase_df1,all_cpls_df1)

In [ ]:
all_cpls_df1

In [ ]:
save_path = result_path + "r2aplus_30degree_cohorte1/replicates2/"
if isdir(save_path) == False:
    mkdir(save_path)
content_df2, min_val, max_val, exp_phase_df2, stat_phase_df2 = plot_content(df2, save_path)

all_cpls_df2 = {}
all_cpls_df2 = collect_interesting_cpl_data(cpls,content_df2,exp_phase_df2,stat_phase_df2,all_cpls_df2)

In [ ]:
all_cpls_df2

In [ ]:
save_path = result_path + "r2aplus_30degree_cohorte1/replicates3/"
if isdir(save_path) == False:
    mkdir(save_path)
    
content_cohorte1, min_val, max_val, exp_phase_cohorte1, stat_phase_cohorte1 = plot_content(df3, save_path)

all_cpls_df3 = {}
all_cpls_df3 = collect_interesting_cpl_data(cpls,content_cohorte1,exp_phase_cohorte1,stat_phase_cohorte1,all_cpls_df3)

# collecting values for df 3
all_cpls = collect_interesting_cpl_data(cpls,content_cohorte1,exp_phase_cohorte1,stat_phase_cohorte1,all_cpls)

In [ ]:
all_cpls

In [ ]:
all_cpls_df3

In [ ]:
df1 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep1.csv",skiprows=11, delimiter=";")
df2 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep2.csv",skiprows=11, delimiter=";")
df3 = pd.read_csv(data_path + "r2aplus_30degree/csv/co2rep3.csv", skiprows=11, delimiter=";")

In [ ]:
savep="../results/CPL_0095_combined_plot.png"
cpl_number = "CPL_0095"
plot_combined_od_red_vs_green_plot(df3,cpl_number,od_color,rfp_gfp_color,vline_color, savep)

In [ ]:
savep="../results/CPL_0109_combined_plot.png"
cpl_number = "CPL_0109"
plot_combined_od_red_vs_green_plot(df3,cpl_number,od_color,rfp_gfp_color,vline_color, savep)

In [ ]:
save_path = result_path + "r2aplus_30degree_cohorte2/replicates1/"
if isdir(result_path + "r2aplus_30degree_cohorte2/") == False:
    mkdir(result_path + "r2aplus_30degree_cohorte2/")
if isdir(save_path) == False:
    mkdir(save_path)
content_cohorte2_df1, min_val, max_val, exp_phase_cohorte2_df1, stat_phase_cohorte2_df1 = plot_content(df1, save_path)

all_cpls_df1_cohorte2 = {}
all_cpls_df1_cohorte2 = collect_interesting_cpl_data(cpls,
                                                     content_cohorte2_df1,
                                                     exp_phase_cohorte2_df1,
                                                     stat_phase_cohorte2_df1,
                                                     all_cpls_df1_cohorte2)

all_cpls_df1_cohorte2

In [ ]:
save_path = result_path + "r2aplus_30degree_cohorte2/replicates2/"
if isdir(save_path) == False:
    mkdir(save_path)
content_cohorte2_df2, min_val, max_val, exp_phase_cohorte2_df2, stat_phase_cohorte2_df2 = plot_content(df2, save_path)

all_cpls_df2_cohorte2 = {}
all_cpls_df2_cohorte2 = collect_interesting_cpl_data(cpls,
                                                     content_cohorte2_df2,
                                                     exp_phase_cohorte2_df2,
                                                     stat_phase_cohorte2_df2,
                                                     all_cpls_df2_cohorte2)
all_cpls_df2_cohorte2

In [ ]:
save_path = result_path + "r2aplus_30degree_cohorte2/replicates3/"
if isdir(save_path) == False:
    mkdir(save_path)
content_cohorte2, min_val, max_val, exp_phase_cohorte2, stat_phase_cohorte2 = plot_content(df3, save_path)
# collecting values for df 3
all_cpls_df3_cohorte2 = {}

all_cpls_df3_cohorte2 = collect_interesting_cpl_data(cpls,content_cohorte2,exp_phase_cohorte2,stat_phase_cohorte2,all_cpls)

all_cpls = collect_interesting_cpl_data(cpls,content_cohorte2,exp_phase_cohorte2,stat_phase_cohorte2,all_cpls)

In [ ]:
all_cpls_df2_cohorte2

In [ ]:
all_cpls

In [ ]:
all_cpls_df1

In [ ]:
combined_cpls = {}
std_cpls = {}
for key in all_cpls_df1.keys():
    if key not in list(combined_cpls.keys()):
        combined_cpls[key] = [(all_cpls_df1[key][0] + all_cpls_df2[key][0] + all_cpls_df3[key][0])/3,(all_cpls_df1[key][1] + all_cpls_df2[key][1] + all_cpls_df3[key][1])/3 ]
        std_cpls[key] = [np.std([all_cpls_df1[key][0],all_cpls_df2[key][0],all_cpls_df3[key][0]]), np.std([all_cpls_df1[key][1],all_cpls_df2[key][1],all_cpls_df3[key][1]])]
        
for key in all_cpls_df1_cohorte2.keys():
    if key not in list(combined_cpls.keys()):
        combined_cpls[key] = [(all_cpls_df1_cohorte2[key][0] + all_cpls_df2_cohorte2[key][0] + all_cpls_df3_cohorte2[key][0])/3,(all_cpls_df1_cohorte2[key][1] + all_cpls_df2_cohorte2[key][1] + all_cpls_df3_cohorte2[key][1])/3 ]
        std_cpls[key] = [np.std([all_cpls_df1_cohorte2[key][0],all_cpls_df2_cohorte2[key][0],all_cpls_df3_cohorte2[key][0]]), np.std([all_cpls_df1_cohorte2[key][1],all_cpls_df2_cohorte2[key][1],all_cpls_df3_cohorte2[key][1]])]

In [ ]:
std_cpls

In [ ]:
combined_cpls

# Plotting Figure 4a

In [ ]:
# exp - stat phase
all_cpls

In [ ]:
data_to_plot = clean_collected_dictionary(all_cpls)

In [ ]:
# 0 = content | 1 = exp | 2 = stat
data_to_plot

In [ ]:
save_path = "../results/"
savep = save_path + "exponential_phase_replicates3_unsorted.png"
plot_phase_bar_plot(data_to_plot[0], data_to_plot[2], data_to_plot[1], savep, xlim=0.8)

In [ ]:
sorted_stat_phase = []
sorted_exp_phase = []
sorted_cont = []
max_promoter_strength = []
# 0 = content | 1 = exp | 2 = stat
for exp, stat, cont in sorted(zip(data_to_plot[1],data_to_plot[2],data_to_plot[0])):
    max_promoter_strength.append(max([exp,stat]))
    sorted_stat_phase.append(stat)
    sorted_exp_phase.append(exp)
    sorted_cont.append(cont)

In [ ]:
sorted_exp_phase

In [ ]:
sorted_stat_phase

In [ ]:
sorted_cont

In [ ]:
'''plot_phase_bar_plot
    
    Function for plotting the sorted stationary and exponential growth phase RFP/GFP values.
    
    :param content - CPL identifier for yticks
        :type list[str]
    :param stat_phase
        :type list[float]
    :param exp_phase
        :type list[float]
    :param savep
        :type str
    
    :returns returncode
        :type int
'''
def plot_phase_bar_plot(content:list,stat_phase:list,exp_phase:list,savep:str, xlim=1,
                        legend_position="best", fsize=15, fsize_text=8, fig_size=(14,10)):

    try:
        if len(content) == len(stat_phase) == len(exp_phase):
            width = 0.4
            ind = np.arange(len(content))
            
            plt.figure(figsize=fig_size)
            bar1 = plt.barh(ind, exp_phase, width, label="RFP/GFP exponential phase", color="blue", edgecolor="black")
            bar2 = plt.barh(ind + width, stat_phase, width, label="RFP/GFP stationary phase", color="orange", edgecolor="black")
            
            for rect in bar1 + bar2:
                if rect.get_width() > 0:
                    if rect.get_width() > 0.0099999999:
                        if rect.get_width() > 0.1:
                            plt.text(rect.get_width() + 0.012, rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)
                        else:
                            plt.text(rect.get_width() + 0.015, rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)

                    else:
                        plt.text(rect.get_width() + 0.015, rect.get_y() - 0.05, f'{np.abs(rect.get_width()):.3f}', ha='center', va='bottom', fontsize=fsize_text)

            plt.xlabel("RFP/GFP fluorescence values", fontsize=fsize)
            plt.ylabel("Curvibacter Promoter Library (CPL) identifier", fontsize=fsize)
            # plt.title("Comparison of CPL-promoter constructs in exponentially and stationary growth phases")
            plt.yticks(ind + width / 2, content, fontsize=fsize-2 )
            plt.xticks(fontsize=20)
            plt.legend(loc=legend_position, fontsize=fsize-2)
            plt.xlim(0,xlim)
            plt.grid(axis="x")
            
            ax = plt.gca()

            # Set the y-tick labels
            ytick_labels = ax.get_yticklabels()
            
            promoters = ["CPL0025","CPL0095","CPL0022","CPL0017"]

            # Set the color for the specified indices
            for label in ytick_labels:
                if label.get_text() in promoters:
                    label.set_color('red')
                    
            
            plt.tight_layout()
            plt.savefig(savep)
            return 0
        else:
            raise Exception("[-] lists do not have the same length: content: {} stat_phase: {}  exp_phase: {}"
                            .format(len(content),len(stat_phase),len(exp_phase)))
    except Exception as e:
        raise Exception("[-] ERROR plotting phase bar plot with exception: {}".format(e))

In [ ]:
save_path = "../results/"
savep = save_path + "exponential_phase_replicates3_sorted.png"
label = plot_phase_bar_plot(sorted_cont,sorted_stat_phase, sorted_exp_phase, savep, xlim=0.8, legend_position="lower right", fsize=20, fsize_text=10, fig_size=(14,14))

# Plotting Mean Value Phase Bar Plot Figure 4a

In [ ]:
# content 0 - exp 1 - stat 2 - std exp 3 - std stat 4
data_to_plot = [[],[],[],[],[]
]

for key in combined_cpls.keys():
    data_to_plot[0].append(key)
    data_to_plot[1].append(combined_cpls[key][0])
    data_to_plot[2].append(combined_cpls[key][1])
    data_to_plot[3].append(std_cpls[key][0])
    data_to_plot[4].append(std_cpls[key][1])
    
sorted_stat_phase = []
sorted_exp_phase = []
sorted_cont = []
sorted_std_exp = []
sorted_std_stat = []
max_promoter_strength_with_mean = []
# 0 = content | 1 = exp | 2 = stat
for exp, stat, cont, std_exp, std_stat in sorted(zip(data_to_plot[1],
                                                     data_to_plot[2],
                                                     data_to_plot[0],
                                                     data_to_plot[3],
                                                     data_to_plot[4])):
    max_promoter_strength_with_mean.append(max([exp,stat]))
    sorted_stat_phase.append(stat)
    sorted_exp_phase.append(exp)
    sorted_cont.append(cont)
    sorted_std_exp.append(std_exp)
    sorted_std_stat.append(std_stat)

In [ ]:
import matplotlib.colors as mcolors

def get_closest_color_name(rgba_color):
    # Convert RGBA to RGB by ignoring the alpha value
    rgba_color = tuple(rgba_color[:3])
    
    min_distance = float('inf')
    closest_color_name = None
    
    for name, hex_value in mcolors.CSS4_COLORS.items():
        # Convert hex to RGBA
        rgba = mcolors.to_rgba(hex_value)
        # Compare RGB values (ignore alpha)
        distance = sum((c1 - c2) ** 2 for c1, c2 in zip(rgba_color, rgba[:3]))
        
        if distance < min_distance:
            min_distance = distance
            closest_color_name = name
    
    return closest_color_name


'''
plot_phase_bar_plot
    
    Function for plotting the sorted stationary and exponential growth phase RFP/GFP values.
    
    :param content - CPL identifier for yticks
        :type list[str]
    :param stat_phase
        :type list[float]
    :param exp_phase
        :type list[float]
    :param savep
        :type str
    
    :returns returncode
        :type int
'''
def plot_phase_bar_plot_mean(content:list,stat_phase:list,exp_phase:list,std_stat:list,std_exp:list,savep:str, xlim=1,
                        legend_position="best", fsize=15, fsize_text=8, fig_size=(14,10)):

    try:
        if len(content) == len(stat_phase) == len(exp_phase):
            width = 0.4
            ind = np.arange(len(content))
            
            plt.figure(figsize=fig_size)
            bar1 = plt.barh(ind, exp_phase, width,xerr=std_exp,capsize=5, label="RFP/GFP exponential phase", color="blue", edgecolor="black")
            bar2 = plt.barh(ind + width, stat_phase,width, xerr=std_stat,capsize=5, label="RFP/GFP stationary phase", color="orange", edgecolor="black")
            
            
            std_list = std_exp + std_stat
            for index, rect in enumerate(bar1 + bar2):
                # print(index, get_closest_color_name(rect.get_facecolor()))
                if rect.get_width() > 0:
                    if rect.get_width() > 0.0099999999:
                        if rect.get_width() > 0.1:
                            
                            if get_closest_color_name(rect.get_facecolor()) == "orange":
                            
                                plt.text(rect.get_width() + 0.012 + std_list[index], rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)
                            else:
                                plt.text(rect.get_width() + 0.012  + std_list[index], rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)
                              
                        
                        else:
                            
                            if get_closest_color_name(rect.get_facecolor()) == "orange":

                            
                                plt.text(rect.get_width() + 0.015 + std_list[index], rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)
                            
                            else:
                                plt.text(rect.get_width() + 0.015 + std_list[index], rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)

                    else:
                        if get_closest_color_name(rect.get_facecolor()) == "orange":
                        
                            plt.text(rect.get_width() + 0.015 + std_list[index], rect.get_y() - 0.05, f'{np.abs(rect.get_width()):.3f}', ha='center', va='bottom', fontsize=fsize_text)
                        else:
                            plt.text(rect.get_width() + 0.015 + std_list[index], rect.get_y() - 0.05, f'{np.abs(rect.get_width()):.3f}', ha='center', va='bottom', fontsize=fsize_text)

                            
            plt.xlabel("RFP/GFP fluorescence values", fontsize=fsize)
            plt.ylabel("Curvibacter Promoter Library (CPL) identifier", fontsize=fsize)
            # plt.title("Comparison of CPL-promoter constructs in exponentially and stationary growth phases")
            plt.yticks(ind + width / 2, content, fontsize=fsize-2 )
            plt.xticks(fontsize=20)
            plt.legend(loc=legend_position, fontsize=fsize-2)
            plt.xlim(0,xlim)
            plt.grid(axis="x")
            
            ax = plt.gca()

            # Set the y-tick labels
            ytick_labels = ax.get_yticklabels()
            
            promoters = ["CPL0025","CPL0095","CPL0022","CPL0017"]

            # Set the color for the specified indices
            for label in ytick_labels:
                if label.get_text() in promoters:
                    label.set_color('red')
                    
            
            plt.tight_layout()
            plt.savefig(savep)
            return rect
        else:
            raise Exception("[-] lists do not have the same length: content: {} stat_phase: {}  exp_phase: {}"
                            .format(len(content),len(stat_phase),len(exp_phase)))
    except Exception as e:
        raise Exception("[-] ERROR plotting phase bar plot with exception: {}".format(e))

In [ ]:
save_path = "../results/"
savep = save_path + "exponential_phase_mean_values_sorted_with_errorbars.png"
label = plot_phase_bar_plot(sorted_cont,sorted_stat_phase, sorted_exp_phase, sorted_std_stat,sorted_std_exp, savep, xlim=0.8, legend_position="lower right", fsize=20, fsize_text=10, fig_size=(14,14))

# Plotting of Figure 4b

In [ ]:
data_to_plot = clean_collected_dictionary(all_cpls)
sorted_stat_phase = []
sorted_exp_phase = []
sorted_cont = []
max_promoter_strength = []
# 0 = content | 1 = exp | 2 = stat
for exp, stat, cont in sorted(zip(data_to_plot[1],data_to_plot[2],data_to_plot[0])):
    max_promoter_strength.append(max([exp,stat]))
    sorted_stat_phase.append(stat)
    sorted_exp_phase.append(exp)
    sorted_cont.append(cont)

In [ ]:
promoter_strength = np.array(sorted_exp_phase) / np.array(sorted_stat_phase)
promoter_strength_sorted = []
sorted_cont_sorted = []
max_promoter_strength_sorted = []
for val, cont, maxprotstr in sorted(zip(promoter_strength, sorted_cont, max_promoter_strength)):
    max_promoter_strength_sorted.append(maxprotstr)
    promoter_strength_sorted.append(val)
    sorted_cont_sorted.append(cont)

In [ ]:
sorted_exp_phase

In [ ]:
sorted_stat_phase

In [ ]:
plt.figure(figsize=(12,8))

msize = np.interp(max_promoter_strength_sorted,  [min(max_promoter_strength_sorted), max(max_promoter_strength_sorted)], [100,1000])

xticks = []
for tick in range(len(sorted_cont_sorted)):
    xticks.append(tick)

plt.scatter(y=promoter_strength_sorted,x=xticks,s=msize, c="grey", edgecolor="black", lw=2, alpha=0.5, label="marker size based on max RFP / GFP values")
plt.axhline(0.7, color="red", linestyle="--", label="0.7 threshold - stronger in stat. phase")
plt.axhline(1.2, color="red", linestyle="-", label="1.2 threshold - stronger in exp. phase")



    
plt.xticks(xticks, sorted_cont_sorted, rotation= 90, fontsize=17)
plt.yticks(fontsize=17)

#for x,y,text in zip(xticks, promoter_strength_sorted, promoter_strength_sorted):
#    plt.text(x, y, str(round(text,2)), fontsize=12)


ax = plt.gca()

# Set the y-tick labels
xtick_labels = ax.get_xticklabels()

promoters = ["CPL0025","CPL0095","CPL0022","CPL0017"]

# Set the color for the specified indices
for label in xtick_labels:
    if label.get_text() in promoters:
        label.set_color('red')


plt.ylim(0.0,2.0)
plt.legend(fontsize=15, loc="upper left")
plt.grid(axis="y", color="black", alpha=0.8)
plt.grid(axis="x", color="black",alpha=0.1, linestyle="--")
plt.ylabel("RFP/GFP in exp. phase / RFP/GFP in stat. phase", fontsize=15)
plt.tight_layout()
plt.savefig("../results/promoter_strength.png", dpi=400)

# Mean Promotor Strength

In [ ]:
# content 0 - exp 1 - stat 2 - std exp 3 - std stat 4
data_to_plot = [[],[],[],[],[]
]

for key in combined_cpls.keys():
    data_to_plot[0].append(key)
    data_to_plot[1].append(combined_cpls[key][0])
    data_to_plot[2].append(combined_cpls[key][1])
    data_to_plot[3].append(std_cpls[key][0])
    data_to_plot[4].append(std_cpls[key][1])
    
sorted_stat_phase = []
sorted_exp_phase = []
sorted_cont = []
sorted_std_exp = []
sorted_std_stat = []
max_promoter_strength_with_mean = []
# 0 = content | 1 = exp | 2 = stat
for exp, stat, cont, std_exp, std_stat in sorted(zip(data_to_plot[1],
                                                     data_to_plot[2],
                                                     data_to_plot[0],
                                                     data_to_plot[3],
                                                     data_to_plot[4])):
    max_promoter_strength_with_mean.append(max([exp,stat]))
    sorted_stat_phase.append(stat)
    sorted_exp_phase.append(exp)
    sorted_cont.append(cont)
    sorted_std_exp.append(std_exp)
    sorted_std_stat.append(std_stat)

In [ ]:
promoter_strength_combined = {}
#promoter_strength_std = {}

promoter_strength = []
promoter_strength_std = []
promoter_cont = []


for key in all_cpls_df1.keys():
    promoter_strength_combined[key] = [all_cpls_df1[key][0]/all_cpls_df1[key][1],
                                        all_cpls_df2[key][0]/all_cpls_df2[key][1],
                                        all_cpls_df3[key][0]/all_cpls_df3[key][1]]
    #promoter_strength_std[key] = np.std(promoter_strength_combined[key])
    
    promoter_strength.append(np.mean( promoter_strength_combined[key]))
    promoter_strength_std.append(np.var( promoter_strength_combined[key]))
    promoter_cont.append(key)

for key in all_cpls_df1_cohorte2.keys():
    if key not in list(promoter_strength_combined.keys()):
        promoter_strength_combined[key] = [all_cpls_df1_cohorte2[key][0]/all_cpls_df1_cohorte2[key][1],
                                            all_cpls_df2_cohorte2[key][0]/all_cpls_df2_cohorte2[key][1],
                                            all_cpls_df3_cohorte2[key][0]/all_cpls_df3_cohorte2[key][1]]
        #promoter_strength_std[key] = np.std(promoter_strength_combined[key])
        
        promoter_strength.append(np.mean(promoter_strength_combined[key]))
        promoter_strength_std.append(np.var(promoter_strength_combined[key]))
        promoter_cont.append(key)

In [ ]:
promoter_strength_std_sorted = []
promoter_strength_sorted = []
sorted_cont_sorted = []

max_promoter_strength_with_mean_sorted = []
# 0 = content | 1 = exp | 2 = stat
for pstrength, cont, std, max_with_mean in sorted(zip(promoter_strength,
                                              promoter_cont,
                                              promoter_strength_std, max_promoter_strength_with_mean)):
    
    max_promoter_strength_with_mean_sorted.append(max_with_mean)
    promoter_strength_sorted.append(pstrength)
    promoter_strength_std_sorted.append(std)
    sorted_cont_sorted.append(cont)

In [ ]:
plt.figure(figsize=(12,8))

msize = np.interp(max_promoter_strength_with_mean_sorted,  [min(max_promoter_strength_with_mean_sorted), max(max_promoter_strength_with_mean_sorted)], [100,1000])

xticks = []
for tick in range(len(sorted_cont_sorted)):
    xticks.append(tick)

plt.errorbar(y=promoter_strength_sorted,x=xticks, yerr=promoter_strength_std_sorted, ecolor="black", capsize=5,lw=2, linestyle="none")
plt.scatter(y=promoter_strength_sorted,x=xticks,s=msize, c="grey", edgecolor="black", lw=2, alpha=0.5, label="marker size based on max RFP / GFP values")

plt.axhline(0.7, color="red", linestyle="--", label="0.7 threshold - stronger in stat. phase")
plt.axhline(1.2, color="red", linestyle="-", label="1.2 threshold - stronger in exp. phase")



    
plt.xticks(xticks, sorted_cont_sorted, rotation= 90, fontsize=17)
plt.yticks(fontsize=17)

#for x,y,text in zip(xticks, promoter_strength_sorted, promoter_strength_sorted):
#    plt.text(x, y, str(round(text,2)), fontsize=12)


ax = plt.gca()

# Set the y-tick labels
xtick_labels = ax.get_xticklabels()

promoters = ["CPL0025","CPL0095","CPL0022","CPL0017"]

# Set the color for the specified indices
for label in xtick_labels:
    if label.get_text() in promoters:
        label.set_color('red')


plt.ylim(0.0,2.0)
plt.legend(fontsize=15, loc="upper left")
plt.grid(axis="y", color="black", alpha=0.8)
plt.grid(axis="x", color="black",alpha=0.1, linestyle="--")
plt.ylabel("RFP/GFP in exp. phase / RFP/GFP in stat. phase", fontsize=15)
plt.tight_layout()
plt.savefig("../results/promoter_strength_with_variance.png", dpi=400)

# BMM9 Dataset

In [ ]:
def plot_corrected_bmm9_content(df1:pd.DataFrame, df2:pd.DataFrame, save_path:str):
   
    time1 = generate_time_in_minutes_list(df1)
    datasets1 = extract_datasets(df1)
    content1 = extract_content(df1)
    time2 = generate_time_in_minutes_list(df2)
    datasets2 = extract_datasets(df2)
    content2 = extract_content(df2)
    
    od_data = datasets1[0].merge(datasets2[0], on="Content")
    green_data = datasets1[1].merge(datasets2[1], on="Content")
    red_data = datasets1[2].merge(datasets2[2], on="Content")
    od_data = od_data.drop('Well_y', axis=1)
    green_data = green_data.drop('Well_y', axis=1)
    red_data = red_data.drop('Well_y', axis=1)

    time = time1
    for t in time2:
        current_val = time[-1]
        current_val += 15
        time.append(current_val)
    content = content1
    
    min_val = []
    max_val = []
    exp_phase = []
    stat_phase = []
    
    for cont in content:
        od_arr = change_values("Content",cont,od_data, od_data.columns[2:])
        green_arr = change_values("Content",cont,green_data, green_data.columns[2:])
        red_arr = change_values("Content",cont,red_data, red_data.columns[2:])
                
        savep = save_path + cont + "_OD600.png"
        index_exp, index_stat = plot_od_array(cont, time, od_arr, savep)
        savep = save_path + cont + "_red_vs_green_sm.png"
        red_vs_green = np.array(red_arr.astype(float) / green_arr.astype(float))
        smoothed_red_vs_green = outlier_smoother(red_vs_green,time, cont, index_exp, index_stat, savep,plots=True)
        savep = save_path + cont + "_red_vs_green_sm_outlier_removed.png"
        plot_red_vs_green(cont, time, smoothed_red_vs_green,index_exp, index_stat, savep)
        
        min_val.append(min(smoothed_red_vs_green))
        max_val.append(max(smoothed_red_vs_green))
        exp_phase.append(red_vs_green[index_exp])
        stat_phase.append(red_vs_green[index_stat])
        
    savep = save_path + "local_maxima.png"
    plot_bar(max_val, content, savep, "Local maxima of RFP/GFP")
    savep = save_path + "local_minima.png"
    plot_bar(min_val, content, savep, "Local minima of RFP/GFP")
    savep = save_path + "exponential_phase.png"
    plot_bar(exp_phase, content, savep, "Values of RFP/GFP in exponential phase")
    savep = save_path + "stationary_phase.png"
    plot_bar(stat_phase, content, savep, "Values of RFP/GFP in stationary phase")
    savep = save_path + "exponential_stationary_phase_bar_plots.png"
    plot_phase_bar_plot(content, stat_phase, exp_phase, savep)
    savep = save_path + "min_max_bar_plots.png"
    plot_min_max_bar_plot(content, min_values=min_val, max_values=max_val, savep=savep)
    dataframe = pd.DataFrame(zip(content, exp_phase, stat_phase, min_val, max_val),
                         columns=["CPL","EXP_PHASE","STAT_PHASE","MIN_VAL","MAX_VAL"])
    dataframe.to_csv(save_path+"comorehensive_dataframe.csv")
    return content, min_val, max_val, exp_phase, stat_phase

In [ ]:
df1 = pd.read_csv(data_path + "bmm9_30degree/csv/co1rep1.csv", skiprows=11, delimiter=";")
df2 = pd.read_csv(data_path + "bmm9_30degree/csv/co1rep1p2.csv",skiprows=11, delimiter=";")

In [ ]:
save_path = result_path + "bmm9_30degree_cohorte1/replicates1/"

if isdir(result_path + "bmm9_30degree_cohorte1/") == False:
    mkdir(result_path + "bmm9_30degree_cohorte1/")

if isdir(save_path) == False:
    mkdir(save_path)

content, min_val, max_val, exp_phase, stat_phase = plot_corrected_bmm9_content(df1, df2, save_path)

In [ ]:
df1 = pd.read_csv(data_path + "bmm9_30degree/csv/co2rep1.csv", skiprows=10, delimiter=";")

In [ ]:
save_path = result_path + "bmm9_30degree_cohorte2/replicates1/"

if isdir(result_path + "bmm9_30degree_cohorte2/") == False:
    mkdir(result_path + "bmm9_30degree_cohorte2/")

if isdir(save_path) == False:
    mkdir(save_path)

content, min_val, max_val, exp_phase, stat_phase = plot_content(df1, save_path)

# calculating turning points

- this is already implemented in the functions above, the savgol_filter does the trick
- however, this section is an example how to approximate the relative growth phases

In [ ]:
df1 = pd.read_csv(data_path + "r2aplus_30degree/csv/cohort1rep1_reconstituted.csv", delimiter="\t")
df2 = pd.read_csv(data_path + "r2aplus_30degree/csv/co1rep2.csv",skiprows=11, delimiter=";")
df3 = pd.read_csv(data_path + "r2aplus_30degree/csv/co1rep3.csv", skiprows=11, delimiter=";")

In [ ]:
# list of dataframes [od, green, red]
dataframes_df2 = extract_datasets(df2)
cpl0025_od = list(dataframes_df2[0].iloc[20][2:].apply(lambda x: float(x.replace(",","."))))

In [ ]:
cpl0012_od = list(dataframes_df2[0].iloc[11][2:].apply(lambda x: float(x.replace(",","."))))

In [ ]:
plt.scatter(x=range(len(cpl0025_od)),y=cpl0025_od)
plt.scatter(x=range(len(cpl0025_od)),y=np.gradient(cpl0025_od))

In [ ]:
plt.scatter(x=range(len(cpl0012_od)),y=cpl0012_od)
plt.scatter(x=range(len(cpl0012_od)),y=np.gradient(cpl0012_od))

In [ ]:
# omit first 20 datapoints and last 20 datapoints
# cpl0012_od = cpl0012_od[20:len(cpl0012_od)-20]
max_slope = [x - z for x, z in zip(cpl0012_od[1:], cpl0012_od[:-1])]

for index,val in enumerate(max_slope):
    if val == max(max_slope[20:-20]):
        print(index, val, max_slope[index])
        break

vlabel = str(index*15) + " min."
plt.scatter(x=range(len(cpl0012_od)),y=cpl0012_od)
plt.vlines(index, 0, max(cpl0012_od), color="red", label=vlabel)
plt.legend()

In [ ]:
max_slope = [x - z for x, z in zip(cpl0025_od[1:], cpl0025_od[:-1])]

for index,val in enumerate(max_slope):
    if val == max(max_slope[20:-20]):
        print(index, val, max_slope[index])
        break

vlabel = str(index*15) + " min."
plt.scatter(x=range(len(cpl0025_od)),y=cpl0025_od)
plt.vlines(index, 0, max(cpl0025_od), color="red", label=vlabel)
plt.legend()

In [ ]:
cpl0025_od = savgol_filter(cpl0025_od,window_length=19,polyorder=2)

In [ ]:
max_slope = [x - z for x, z in zip(cpl0025_od[1:], cpl0025_od[:-1])]

for index,val in enumerate(max_slope):
    if val == max(max_slope):
        print(index, val, max_slope[index])
        break

vlabel = str(index*15) + " min."
plt.scatter(x=range(len(cpl0025_od)),y=cpl0025_od)
plt.vlines(index, 0, max(cpl0025_od), color="red", label=vlabel)
plt.legend()

# plot raw read counts vs CPL

In [ ]:
raw_counts = "../data/readCountsNormalizedWpToAEP.csv"
#raw_counts = "../data/raw_read_counts.csv"
raw_counts_df = pd.read_csv(raw_counts)

In [ ]:
raw_counts_df.head()

In [ ]:
cpl_to_aep = "../data/cpls_to_aep.csv"
cpls_to_aep_df = pd.read_csv(cpl_to_aep)

In [ ]:
colnames = list(cpls_to_aep_df.columns)
colnames[-1] = "aep"
cpls_to_aep_df.columns = colnames
cpls_to_aep_df.columns

In [ ]:
cpls_to_aep_df.head()

In [ ]:
# we need to translate the old aep number to the new aep_rs number ...
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [ ]:
data_dir="../data/"
gbfile=data_dir+'GCF_002163715.1_ASM216371v1_genomic.gbff'

records = []
for record in SeqIO.parse(gbfile,'genbank'):
    records.append(record)

In [ ]:
old_to_new_locus_tag = {}
for record in records:
    for feature in record.features:   
        if "old_locus_tag" in feature.qualifiers.keys():
            
            old_to_new_locus_tag[feature.qualifiers["old_locus_tag"][0]] = feature.qualifiers["locus_tag"][0]

In [ ]:
raw_counts_df["aep"] = raw_counts_df['aep'].apply(lambda x: x if x not in old_to_new_locus_tag.keys() else old_to_new_locus_tag[x])

In [ ]:
merged_df = cpls_to_aep_df.merge(raw_counts_df,on="aep")
merged_df.head()

In [ ]:
merged_df['exponential_phase'] = merged_df['strain ID'].apply(lambda x: 0.0 if x not in all_cpls.keys() else all_cpls[x][0])
merged_df['stationary_phase'] = merged_df['strain ID'].apply(lambda x: 0.0 if x not in all_cpls.keys() else all_cpls[x][1])

In [ ]:
merged_df

In [ ]:
read_counts = []
strain_id = []
exp_phase = []
stat_phase = []

for rc, si, ep, sp in zip(merged_df["readCountMeanGSamples"],merged_df["strain ID"], merged_df["exponential_phase"], merged_df["stationary_phase"]):
    if ep != 0.0 and sp != 0.0:
        read_counts.append(rc)
        strain_id.append(si)
        exp_phase.append(ep)
        stat_phase.append(sp)
        


In [ ]:
# + 1 because of CPL_0017
len(read_counts) + 1 == len(all_cpls.keys())

In [ ]:
# normalize values to be between 0 and 1
read_counts_norm = (np.array(read_counts) - min(read_counts))/ (max(read_counts) - min(read_counts))
exp_phase_norm = (np.array(exp_phase) - min(exp_phase))/ (max(exp_phase) - min(exp_phase))
stat_phase_norm = (np.array(stat_phase) - min(stat_phase))/ (max(stat_phase) - min(stat_phase))

In [ ]:
def plot_norm_vs_flur_bar_plot(content:list,norm_counts:list,phase:list,savep:str, xlim=1, legend_position="best", fsize=15, fsize_text=8, fig_size=(14,10)):

    try:
        if len(content) == len(stat_phase) == len(exp_phase):
            width = 0.4
            ind = np.arange(len(content))
            
            plt.figure(figsize=fig_size)
            bar1 = plt.barh(ind, norm_counts, width, label="Scaled Mean Read Counts", color="blue", edgecolor="black")
            bar2 = plt.barh(ind + width, phase, width, label="Scaled RFP/GFP values in exp. phase", color="orange", edgecolor="black")
            
            for rect in bar1 + bar2:
                if rect.get_width() > 0:
                    if rect.get_width() > 0.0099999999:
                        if rect.get_width() > 0.1:
                            plt.text(rect.get_width() + 0.022, rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)
                        else:
                            plt.text(rect.get_width()+ 0.022, rect.get_y(), f'{np.abs(rect.get_width()):.2f}', ha='center', va='bottom', fontsize=fsize_text)

                    else:
                        plt.text(rect.get_width() + 0.022, rect.get_y(), f'{np.abs(rect.get_width()):.3f}', ha='center', va='bottom', fontsize=fsize_text)

            plt.ylabel("Curvibacter Promoter Library (CPL) identifier", fontsize=fsize)
            # plt.title("Comparison of CPL-promoter constructs in exponentially and stationary growth phases")
            plt.yticks(ind + width / 2, content, fontsize=fsize-2)
            #plt.xlabel("RFP/GFP Values in exp. phase\n Scaled (0-1) Mean Read Counts", fontsize=17)
            ax = plt.gca()

            # Set the y-tick labels
            ytick_labels = ax.get_yticklabels()
            
            promoters = ["CPL0025","CPL0095","CPL0022","CPL0017"]

            # Set the color for the specified indices
            for label in ytick_labels:
                if label.get_text() in promoters:
                    label.set_color('red')
            plt.xticks(fontsize=20)
            plt.legend(loc=legend_position, fontsize=fsize-2)
            plt.tight_layout()
            plt.xlim(0,xlim)
            plt.grid(axis="x")
            plt.savefig(savep)
            #plt.close()
        else:
            raise Exception("[-] lists do not have the same length: content: {} stat_phase: {}  exp_phase: {}"
                            .format(len(content),len(stat_phase),len(exp_phase)))
    except Exception as e:
        raise Exception("[-] ERROR plotting phase bar plot with exception: {}".format(e))

In [ ]:
savep = "../results/normalizedCounts_vs_exp_phase.png"

sorted_exp_phase = []
sorted_read_counts = []
sorted_cont = []
sorted_non_norm_exp_phase = []
sorted_non_norm_read_counts = []


for exp, counts, cont, non_norm_exp, non_norm_counts in sorted(zip(exp_phase_norm,read_counts_norm,strain_id, exp_phase, read_counts)):
    sorted_read_counts.append(counts)
    sorted_exp_phase.append(exp)
    sorted_cont.append(cont)
    sorted_non_norm_exp_phase.append(non_norm_exp)
    sorted_non_norm_read_counts.append(non_norm_counts)
    
plot_norm_vs_flur_bar_plot(sorted_cont, sorted_read_counts, sorted_exp_phase, savep, xlim=1.2, legend_position="center right", fsize=20, fsize_text=10, fig_size=(14,14))

In [ ]:
savep = "../results/normalizedCounts_vs_stat_phase.png"
plot_norm_vs_flur_bar_plot(strain_id, read_counts_norm, stat_phase_norm, savep, xlim=1.2, legend_position="lower right", fsize=20, fsize_text=10, fig_size=(14,14))

In [ ]:
# none normalized (0-1) lists of exponential phase RFU values (RFP/GFP) and nroamlized read count data
exp_phase_log = []
read_counts_log = []
cpls = []

for exp, count, cpl in sorted(zip(exp_phase, read_counts,strain_id)):
    
    if exp != 0.0 and count != 0.0:
        
        exp_phase_log.append(np.log(exp))
        read_counts_log.append(np.log(count))
        cpls.append(cpl)
    else:
        if exp == 0.0:
            exp = 0.0000001
        if count == 0.0:
            count = 0.0000001
            
        exp_phase_log.append(np.log(exp))
        read_counts_log.append(np.log(count))
        cpls.append(cpl)  

savep = "../results/exp_phase_log_vs_read_counts_log.png"

# Plotting figure 6 - correlation of RNAseq read data and measured promoter strength

In [ ]:
r_pearson, p_pearson = pearsonr(exp_phase_log, read_counts_log)

In [ ]:
r_spearman, p_spearman = spearmanr(exp_phase_log, read_counts_log)

In [ ]:
msize = np.interp(sorted_non_norm_exp_phase,  [min(sorted_non_norm_exp_phase), max(sorted_non_norm_exp_phase)], [100,1000])

colormap = plt.get_cmap('viridis')

# Normalize the data to map it to colormap range
norm = plt.Normalize(min(sorted_non_norm_read_counts), max(sorted_non_norm_read_counts))

# hex_colors = [colormap(norm(value)) for value in sorted_non_norm_read_counts]
# hex_colors = [plt.cm.colors.rgb2hex(color) for color in hex_colors]

plt.figure(figsize=(12,8))
# exp_phase_log, poly1d_fn(exp_phase_log), '--k', 
# plt.scatter(exp_phase_log,read_counts_log, 
#         s=msize,c=hex_colors, edgecolor="black", alpha=0.8)

plt.scatter(exp_phase_log,read_counts_log, edgecolor="black", alpha=0.8)

# optimal linear fit
#x1, y1 = [min(exp_phase_log), max(exp_phase_log)], [min(read_counts_log), max(read_counts_log)]
#plt.plot(x1, y1,'--k', alpha=0.6, label="optimal linear fit") #label="f(x,y) = x,y"

# calculating regression line
m, b = np.polyfit(exp_phase_log, read_counts_log, 1)
plt.plot(exp_phase_log, m*np.array(exp_phase_log)+b, '--b', label="regression line")


plt.xlabel("log10-transformed RFP/GFP values of mid-exponential growth phase", fontsize=15)
plt.ylabel("log10-transformed RNAseq mean read counts", fontsize=15)
#locs, labels = plt.xticks()
plt.grid()

for x,y,text in zip(exp_phase_log, read_counts_log, cpls):
    if text == "CPL0001":
        plt.text(x-0.05, y+.08, text, fontsize=12)
    elif text == "CPL0008":
        plt.text(x+0.07, y, text, fontsize=12)
    elif text == "CPL0005":
        plt.text(x+0.1, y-0.05, text, fontsize=12)

    elif text == "CPL0071":
        plt.text(x-.03, y-.22, text, fontsize=12)
    elif text == "CPL0112":
        plt.text(x+.03, y+.12, text, fontsize=12)
    elif text == "CPL0095" or text == "CPL0022" or text == "CPL0025":
        plt.text(x+.07, y+.08, text, fontsize=12, color="red")
    else:
        plt.text(x+.07, y+.08, text, fontsize=12)
#plt.xticks(exp_phase_log,cpls, rotation=95)

plt.text(-6, 7.5, "Pearson R: {} \np-value: {}".format(round(r_pearson,2), round(p_pearson,2)), color="blue", fontsize=12)


#colorbar = plt.colorbar(plt.cm.ScalarMappable(cmap=colormap, norm=norm))
#colorbar.set_label("mean read counts", fontsize=15) 


plt.xlim(-7.2,-0.5)
plt.tight_layout()
plt.legend(loc="lower right", fontsize=15)



plt.savefig(savep, dpi=400)

# calculate GC content

In [ ]:
old_to_new_locus_tag

In [ ]:
highlight_sequences = {}
with open("../data/target_promoter_sequences.faa","r") as targetfile:
    for line in targetfile.readlines():
        if line.startswith(">"):
            aep = line.rstrip().split(">")[1]
            if aep in old_to_new_locus_tag.keys():
                aep = old_to_new_locus_tag[aep]
            highlight_sequences[aep] = ""
        else:
            highlight_sequences[aep] += line.rstrip()

In [ ]:
for promoter in highlight_sequences.keys():
    gc_content = (highlight_sequences[promoter].count("G") + highlight_sequences[promoter].count("C")) / len(highlight_sequences[promoter])
    print(promoter, gc_content, len(highlight_sequences[promoter]))

# Supplementary Table 3 

In [ ]:
sites = pd.read_csv("../data/xtreme_output/sites.tsv", delimiter="\t")

In [ ]:
sites.head()

In [ ]:
sites['new_AEP'] = sites.seq_ID.apply(lambda x: old_to_new_locus_tag[x] if x in old_to_new_locus_tag.keys() else x)

In [ ]:
sites.head()

In [ ]:
aep_to_cpl = pd.read_csv("../data/cpls_to_aep.csv")

In [ ]:
aep_to_cpl_dict = {}
for strain, aep in zip(aep_to_cpl['strain ID'],aep_to_cpl['AEP nr']):
    aep_to_cpl_dict[aep] = strain

In [ ]:
aep_to_cpl_dict["AEP_RS12495"] = "CPL0073"

In [ ]:
aep_to_cpl_dict

In [ ]:
sites['CPL'] = sites.new_AEP.apply(lambda x: aep_to_cpl_dict[x] if x in aep_to_cpl_dict.keys() else x)

In [ ]:
sites[sites.seq_ID == "AEP_02249"]

In [ ]:
sMotifs = {}
sMotifs["1"] = "AmrZ,LasR: P. aeruginosa; BldD: S. coelicolor"
sMotifs["2"] = "CodY: B. anthracis"
sMotifs["4"] = "LexA: V.parahaemolyticus"
sMotifs["5"] = "ExpR: S. meliloti; DosR: M.tuberculosis"
sMotifs["3"] = "CodY: S.pyogenes; CodY: L.lactis; CcpA: S.pneumoniae"

In [ ]:
excel_savep = "../results/tf_binding_site_motifs.xlsx"

In [ ]:
for cpl in all_cpls.keys():
    if cpl not in list(sites[['CPL','motif_ID','site_Start','site_End','site_Sequence']].CPL):
        print(cpl)

In [ ]:
sites_t = sites[['CPL','motif_ID','site_Start','site_End','site_Sequence']].copy()

In [ ]:
sites[['CPL','motif_ID','site_Start','site_End','site_Sequence']].head()

In [ ]:
sites_t = sites[['CPL','motif_ID','site_Start','site_End','site_Sequence']].copy()
sites_t['similarMotifs'] = sites_t.motif_ID.apply(lambda x: sMotifs[x.split("-")[0]])
sites_t.sort_values(by="CPL",inplace=True)
sites_t.to_excel(excel_savep)